# Tensorflow 불러오기

In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

## Hide deprecated warning
tf.logging.set_verbosity(tf.logging.ERROR)

## Load dataset --> mnist
mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


# 이미지를 입력 받을 공간 & 결과값을 받을 공간 생성

In [2]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 256개의 뉴런으로 구성된 레이어1를 생성
## 초기 입력 공간인 X와는 곱셈관계를 구성

In [3]:
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.1))
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, keep_prob)

# 256개의 뉴런으로 구성된 레이어2를 생성
## 레이어1과 곱셈관계를 구성

In [4]:
W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))
L2 = tf.nn.dropout(L2, keep_prob)

# 10개의 뉴런으로 구성된 레이어를 생성
## 레이어2와 곱셈관계를 구성

In [5]:
W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

# 오류 처리 방식을 선정
## adam 최적화 기법 사용
- http://i.imgur.com/NKsFHJb.gif?1

### 학습 조정 기법에 대한 설명
- http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html

In [21]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# TensorFlow 초기화
## TensorFlow session 생성

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 한번에 100개의 이미지를 학습
## 총 30세트의 전체 이미지 학습

In [ ]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
for epoch in range(30):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))

# 학습 결과 검토
## 정확성 측정

In [9]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Acc:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

Acc: 0.9828


# 결과 시각화
## 학습한 모델이 이미지에 대해 어떤 결과를 도출했는지 확인

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
labels = sess.run(model, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1})
fig = plt.figure()
for i in range(25):
    subplot = fig.add_subplot(5, 5, i + 1)
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title('%d' % np.argmax(labels[i]), fontsize=22)
    subplot.imshow(mnist.test.images[i].reshape((28, 28)), cmap=plt.cm.gray_r)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=1)
plt.show()

# 번외편
## Keras를 적용시킨 최신 TF로 MNIST 실행

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)